---
title: "pheno heads up (blast) table" 
format: html
execute:
    eval: true
jupyter: IRkernel
---

In [1]:
library(jsonlite)
library(DT)
library(IRdisplay)
library(htmlwidgets)

In [2]:
config_dir <- "../phenoRankeR/inst/extdata/config"
json_data <- fromJSON(file.path(config_dir, "pheno_blast_col_colors.json"))

In [11]:
user <- "pheno.ranker@playground.com"
run_id <- "20241006015118"

user_dir <- file.path("../data/user_data", user)
out_dir <- file.path(user_dir, "output/rankedPatients", run_id)
file_suffix <- "_alignment.target.csv"

file_path <- file.path(
  out_dir,
  paste0(run_id, file_suffix)
)
print(file_path)

alignment_df <- read.table(
  file_path,
  header = TRUE,
  sep = ";",
  row.names = NULL
)
head(alignment_df)

[1] "../data/user_data/pheno.ranker@playground.com/output/rankedPatients/20241006015118/20241006015118_alignment.target.csv"


,id,ref,indicator,tar,weight,hamming.distance,json.path,label
,<chr>,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>
1,HG00100,0,,0,1,0,diseases.ICD10:D50.diseaseCode.id.ICD10:D50,iron deficiency anaemia
2,HG00100,0,,0,1,0,diseases.ICD10:E06.diseaseCode.id.ICD10:E06,thyroiditis
3,HG00100,0,,0,1,0,diseases.ICD10:J40.diseaseCode.id.ICD10:J40,acute bronchitis
4,HG00100,0,,0,1,0,diseases.ICD10:J45.diseaseCode.id.ICD10:J45,asthma
5,HG00100,0,,0,1,0,diseases.ICD10:K02.diseaseCode.id.ICD10:K02,dental caries
6,HG00100,0,,0,1,0,ethnicity.id.NCIT:C16352,Black or Black British


In [13]:
pat_id <- "HG00100"

# alignment_df
filtered_df <- alignment_df[alignment_df[, 1] == pat_id, ]

# remove the first column
filtered_df <- filtered_df[, -1]
colnames(filtered_df) <- c(
  "Reference", "Indicator",
  "Target", "Weight",
  "Hamming Distance", "JSON Path", "Label"
)
head(filtered_df)

,Reference,Indicator,Target,Weight,Hamming Distance,JSON Path,Label
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>
1,0,,0,1,0,diseases.ICD10:D50.diseaseCode.id.ICD10:D50,iron deficiency anaemia
2,0,,0,1,0,diseases.ICD10:E06.diseaseCode.id.ICD10:E06,thyroiditis
3,0,,0,1,0,diseases.ICD10:J40.diseaseCode.id.ICD10:J40,acute bronchitis
4,0,,0,1,0,diseases.ICD10:J45.diseaseCode.id.ICD10:J45,asthma
5,0,,0,1,0,diseases.ICD10:K02.diseaseCode.id.ICD10:K02,dental caries
6,0,,0,1,0,ethnicity.id.NCIT:C16352,Black or Black British


In [24]:
json_path_col <- filtered_df$`JSON Path`
# json_path_col

# remove everything after the first dot
top_levels <- gsub("\\..*", "", json_path_vector)
# print(top_levels)

input_format <- "bff"
color_map <- json_data[[input_format]]
# color_map

color_col <- unlist(lapply(top_levels, function(x) color_map[[x]]))

# add the color col to the dataframe
filtered_df$color <- color_col
head(filtered_df)

,Reference,Indicator,Target,Weight,Hamming Distance,JSON Path,Label,color
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>
1,0,,0,1,0,diseases.ICD10:D50.diseaseCode.id.ICD10:D50,iron deficiency anaemia,"rgb(158, 1, 66)"
2,0,,0,1,0,diseases.ICD10:E06.diseaseCode.id.ICD10:E06,thyroiditis,"rgb(158, 1, 66)"
3,0,,0,1,0,diseases.ICD10:J40.diseaseCode.id.ICD10:J40,acute bronchitis,"rgb(158, 1, 66)"
4,0,,0,1,0,diseases.ICD10:J45.diseaseCode.id.ICD10:J45,asthma,"rgb(158, 1, 66)"
5,0,,0,1,0,diseases.ICD10:K02.diseaseCode.id.ICD10:K02,dental caries,"rgb(158, 1, 66)"
6,0,,0,1,0,ethnicity.id.NCIT:C16352,Black or Black British,"rgb(213, 62, 79)"


In [42]:
rowCallback <- JS(
  "function(row, data, index) {
    var color = data[data.length - 1] || 'rgb(255,255,255)'; 
    $('td', row).css('background-color', color) ;
  }"
)

# hide the last column using a initComplete callback
initComplete <- JS(
  "function(settings, json) {
    var table = settings.oInstance.api();
    console.log(table);
    table.column(8).visible(false);
  }"
)

dt <- datatable(
  filtered_df,
  escape = FALSE,
  options = list(
    autoWidth = TRUE,
    columnDefs = list(
      list(
        targets = "_all",
        width = "5px"
      )
    ),
    paging = FALSE,
    searching = TRUE,
    info = FALSE,
    scrollX = TRUE,
    scrollY = "500px",
    rowCallback = rowCallback,
    initComplete = initComplete
  )) %>% formatStyle(
    columns = 6, # Apply style to the JSON Path column
    `white-space` = "nowrap",
    `overflow` = "hidden",
    `text-overflow` = "ellipsis",
    `max-width` = "300px", # Restrict the cell's width
    `cursor` = "pointer"
  )

saved_wiget_path <- "notebooks/saved_widgets"
widget_fn <- "blast.html"

saveWidget(
  dt,
  file.path(
    saved_wiget_path,
    widget_fn
  )
)


In [43]:
# make sure that vscode is running the live server
live_server_path <- "http://localhost:5500/jupyrenv/notebooks/saved_widgets"
iframe_src <- paste0(
  live_server_path,
  "/",
  widget_fn
)

display_html(
  paste0(
    "<iframe src='",
    iframe_src,
    "' width='100%' height='500px'></iframe>"
  )
)

In [33]:
# JavaScript to prepend the header row
headerCallback <- JS(
  "function(thead, data, start, end, display) {",
  sprintf("$(thead).closest('thead').prepend('<tr><th></th>%s</tr>');", header_row),
  "}"
)


# JavaScript to color the columns based on the values in the 2nd row of the data
initComplete <- JS(
  "function(settings, json) {",
  "$('tbody tr').each(function() {",
    # Loop over each column to set background color based on the value in the first row (background-color)
    "for (var i = 0; i < $('tbody tr:eq(0) td').length; i++) {",
      "var color = $('tbody tr:eq(0)').find('td:eq(' + i + ')').text();",
      "console.log(color);",

      # Apply background color to the cell
      "$('tbody tr').find('td:eq(' + i + ')').css('background-color', color);",
      "$('thead th.sorting').eq(i).css('background-color', color);",  # Apply background color to the header

      # hide the row with the colors
      "$('tbody tr').eq(0).hide();",
    "}",
  "});",
  "}"
)

# remove row 2 and 3
bin_df3 <- bin_df2[-c(2, 3),]

dt <- datatable(
  bin_df3,
  rownames = FALSE,
  escape = FALSE,
  options = list(
    headerCallback = headerCallback,
    initComplete = initComplete
  )
)

saved_wiget_path <- "notebooks/saved_widgets"
widget_fn <- "blast.html"

saveWidget(
  dt,
  file.path(
    saved_wiget_path,
    widget_fn
  )
)


ERROR: Error: object 'header_row' not found


In [32]:
# make sure that vscode is running the live server
live_server_path <- "http://localhost:5500/jupyrenv/notebooks/saved_widgets"
iframe_src <- paste0(
  live_server_path,
  "/",
  widget_fn
)

display_html(
  paste0(
    "<iframe src='",
    iframe_src,
    "' width='100%' height='500px'></iframe>"
  )
)